In [45]:
import requests

url="https://v6.exchangerate-api.com/v6/22dd33f5efd4cebd27af814d/pair/USD/BDT"
response = requests.get(url)
data = response.json()
data

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1754769601,
 'time_last_update_utc': 'Sat, 09 Aug 2025 20:00:01 +0000',
 'time_next_update_unix': 1754773201,
 'time_next_update_utc': 'Sat, 09 Aug 2025 21:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'BDT',
 'conversion_rate': 121.55656536}

In [46]:
from langchain_core.tools import tool

@tool
def get_exchange_rate(from_currency: str, to_currency: str) -> float:
    """
    Get the exchange rate from one currency to another.
    """
    url = f"https://v6.exchangerate-api.com/v6/22dd33f5efd4cebd27af814d/pair/{from_currency}/{to_currency}"
    response = requests.get(url)
    data = response.json()
    return data["conversion_rate"]

@tool
def convert_currency(amount: float, conversion_rate: float) -> float:
    """
    Convert currency using conversion rate.
    """
    return amount * conversion_rate


In [47]:
!ollama list

NAME                           ID              SIZE      MODIFIED    
gemma3:4b                      a2af6cc3eb7f    3.3 GB    4 hours ago    
llama3-groq-tool-use:latest    36211dad2b15    4.7 GB    7 hours ago    
mistral:7b                     6577803aa9a0    4.4 GB    2 days ago     
mxbai-embed-large:latest       468836162de7    669 MB    3 days ago     
nomic-embed-text:latest        0a109f422b47    274 MB    3 days ago     
gemma2:2b                      8ccf136fdd52    1.6 GB    4 days ago     
gemma3:1b                      8648f39daa8f    815 MB    4 days ago     
llama3.2:latest                a80c4f17acd5    2.0 GB    4 days ago     
qwen3:4b                       2bfd38a7daaf    2.6 GB    4 days ago     
deepseek-r1:8b                 6995872bfe4c    5.2 GB    6 days ago     


In [48]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2:latest",
    temperature=0.7
)

In [49]:
llm_with_tools=llm.bind_tools([get_exchange_rate, convert_currency])

In [50]:
from langchain_core.messages import HumanMessage

messages=[HumanMessage("What is the exchange rate from USD to BDT and using that rate, convert 10 USD to BDT")]

In [51]:
ai_response = llm_with_tools.invoke(messages)

In [52]:
ai_response

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-08-09T20:04:07.2261701Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5476792000, 'load_duration': 42889800, 'prompt_eval_count': 249, 'prompt_eval_duration': 1555142100, 'eval_count': 64, 'eval_duration': 3877147300, 'model_name': 'llama3.2:latest'}, id='run--1500b9ab-5aa2-4648-9b03-a85f4a643952-0', tool_calls=[{'name': 'get_exchange_rate', 'args': {'from_currency': 'USD', 'to_currency': 'BDT'}, 'id': 'fd6ea390-2cf9-47d2-b2e4-8ae7c0d8b3a6', 'type': 'tool_call'}, {'name': 'convert_currency', 'args': {'amount': '10', 'conversion_rate': {'rate': '1 USD = 85.20 BDT'}}, 'id': '231564fd-33fb-4f7c-b7e8-25ca44c59ab8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 249, 'output_tokens': 64, 'total_tokens': 313})

In [53]:
ai_response.tool_calls

[{'name': 'get_exchange_rate',
  'args': {'from_currency': 'USD', 'to_currency': 'BDT'},
  'id': 'fd6ea390-2cf9-47d2-b2e4-8ae7c0d8b3a6',
  'type': 'tool_call'},
 {'name': 'convert_currency',
  'args': {'amount': '10', 'conversion_rate': {'rate': '1 USD = 85.20 BDT'}},
  'id': '231564fd-33fb-4f7c-b7e8-25ca44c59ab8',
  'type': 'tool_call'}]

#### We want to fetch the conversion rate from Internet! Not auto input by LLM

In [54]:
from langchain_core.tools import tool
from typing import Annotated
from langchain_core.tools import InjectedToolArg


@tool
def get_exchange_rate(from_currency: str, to_currency: str) -> float:
    """
    Get the exchange rate from one currency to another.
    """
    url = f"https://v6.exchangerate-api.com/v6/22dd33f5efd4cebd27af814d/pair/{from_currency}/{to_currency}"
    response = requests.get(url)
    data = response.json()
    return data

#Using InjectedToolArg the LLM will skip inserting unwanted value into conversion_rate variable and will wait for it to be provided during execution.
@tool
def convert_currency(amount: float, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """
    Convert currency using conversion rate.
    """
    return amount * conversion_rate

llm_with_tools=llm.bind_tools([get_exchange_rate, convert_currency])

In [55]:
rate= get_exchange_rate.invoke({"from_currency": "USD", "to_currency": "BDT"})
rate["conversion_rate"]

121.55656536

# Fixed auto input rate

In [59]:
from langchain_core.messages import HumanMessage

messages=[HumanMessage("What is the exchange rate from USD to BDT and using that rate, convert 10 USD to BDT")]
ai_response = llm_with_tools.invoke(messages)
messages.append(ai_response)
ai_response.tool_calls

[{'name': 'get_exchange_rate',
  'args': {'from_currency': 'USD', 'to_currency': 'BDT'},
  'id': '411120de-f9b6-4880-a5f6-0aeefdd59215',
  'type': 'tool_call'},
 {'name': 'convert_currency',
  'args': {'amount': '10'},
  'id': '8afd96ce-8d5b-4b5b-81fe-3ad892c548bb',
  'type': 'tool_call'}]

In [60]:
import json

for tool_call in ai_response.tool_calls:
    if tool_call["name"]=="get_exchange_rate":
        tool_message1=get_exchange_rate.invoke(tool_call)
        exchange_rate=json.loads(tool_message1.content)["conversion_rate"]
        messages.append(tool_message1)
    if tool_call["name"]=="convert_currency":
        tool_call["args"]["conversion_rate"] = exchange_rate
        tool_message2=convert_currency.invoke(tool_call)
        messages.append(tool_message2)

In [61]:
messages

[HumanMessage(content='What is the exchange rate from USD to BDT and using that rate, convert 10 USD to BDT', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-08-09T20:04:49.8516495Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3196473600, 'load_duration': 68812900, 'prompt_eval_count': 236, 'prompt_eval_duration': 97096400, 'eval_count': 44, 'eval_duration': 3028784500, 'model_name': 'llama3.2:latest'}, id='run--e999c339-157d-42e9-9e88-6fa466d5f742-0', tool_calls=[{'name': 'get_exchange_rate', 'args': {'from_currency': 'USD', 'to_currency': 'BDT'}, 'id': '411120de-f9b6-4880-a5f6-0aeefdd59215', 'type': 'tool_call'}, {'name': 'convert_currency', 'args': {'amount': '10', 'conversion_rate': 121.55656536}, 'id': '8afd96ce-8d5b-4b5b-81fe-3ad892c548bb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 236, 'output_tokens': 44, 'total_tokens': 280}),
 ToolMessage(conte

In [62]:
answer=llm_with_tools.invoke(messages)

In [64]:
answer.content

'The current exchange rate from USD to BDT is 1 USD = 121.55656536 BDT.\n\nUsing this rate, we can convert 10 USD to BDT as follows:\n\n10 USD x 121.55656536 BDT/USD = 1215.5656536 BDT'